# Build and Deploy a Fraud Detection Model in Jupyternotebook

In this notebook, we will be working with some auto insurance data and create a predictive model that predicts if an insurance claim is fraudulent or not. This is a Binary Classification task, and we will be creating a Random Forest model using Pyspark.

You will learn to:
    
1. ingest/export data from/to Object Storage in Watson Studio,
2. perform exploratory data analysis(EDA) on the insurance data by drawing some visualizations,
3. preprocess data,
4. create a Random Forest Model using PySpark,
6. Deploy model to Watson Machine Learning.

Note: this analysis is similified to walk you through steps it takes to build and deploy a ML model. You can perform more transformations and analysis on the data once you get the hang of it.

<img src="https://github.com/CatherineCao2016/pics/raw/master/icpdnotebookworkflow.png" align="middle"/>

## Table of Content

1. Import packages
2. Load Insurance data from Object Storage
3. Review Data
4. Data Preparation
5. Exploratory Data Analysis
        5.1 How many fraudulent cases are there in the dataset?
        4.2 5.2 Which incident_severity level has highest fraud ratio?
6. Modeling
7. Save output back to Object Storage
8. Deploy Model to Watson Machine Learning
        

Task list:

    Task 1: Run the cell to import all the packages
    Task 2: Upload sample data to Object Storage and Insert to code
    Task 3: Check summary statistics of df_data_1
    Task 4: Run the cell below to check unqiue values for categorical columns.
    Task 5: Run the cell below to change "?" to "UNKNOWN" for the police_report_available column.
    Task 6: Run the cell below to draw a bar chart of fraud_reported.
    Task 7: Run the cell below to draw a mosaic plot for fraud_reported by incident_severity level.
    Task 8: Run the cell below to build the model. The comment will help you understand what each step is doing.
    Task 9: Follow the steps below and save output file to Object Storage.
    Task 10: Follow the steps below to save model to Watson Machine Learning.

## 1. Import packages

Python packages contain modules, which are simply Python source files that can expose classes, functions and global variables. Users need to import those packages/modules first before you can use those functions in the current environment.

**Task 1: Run the cell to import all the packages**


<div class="panel-group" id="accordion-13">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-13" href="#collapse1-13">
        Hint for running the cell below</a>
      </h4>
    </div>
    <div id="collapse1-13" class="panel-collapse collapse">
      <div class="panel-body"> 
      1. Select the cell below, and press Shift + Return; OR<br>
      2. Select the cell below, and click "Cell" tab in the menu bar => Run Cells.
  </div>

In [ ]:
# for visulization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.mosaicplot import mosaic

# for modeling
from pyspark.sql import SQLContext,SparkSession
from pyspark.sql.types import StringType, DoubleType, FloatType, IntegerType
from pyspark.sql.functions import udf, length,col
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# for model deployment
from watson_machine_learning_client import WatsonMachineLearningAPIClient
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact
from repository.mlrepository import MetaProps

## 2. Load Insurance data from Object Storage(default data repository for Watson Studio)

Watson Studio supports multiple data sources. See a complete list of supported data sources and connectors [here](https://dataplatform.cloud.ibm.com/docs/content/wsj/manage-data/conn_types.html).

**Task 2: Upload sample data to Object Storage and Insert to code**


<form> 
<input type="checkbox"><label>1. Click 1001 botton on the upper right corner, click Files => browse => locate "insurance_claims.csv" you just downloaded to upload it to Object Storage </label> <br>
<input type="checkbox"><label>2. Once it has been uploaded, select the empty cell below then click on "Insert to code" under insurance_claims.csv => choose "Insert Pandas DataFrame" => Run the Cell </label> <br>

</form>

<img src="https://github.com/CatherineCao2016/pics/raw/master/1001.png" width="500" height="800" align="middle"/>

## 3. Review Data

Let's take a quick look at the data by checking summary statistics and unique values of the categorical columns and see if there is any data preparation steps need to be applied to the data.

**Task 3: Check summary statistics of df_data_1**

<div class="panel-group" id="accordion-4">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-4" href="#collapse1-4">
        Hints</a>
      </h4>
    </div>
    <div id="collapse1-4" class="panel-collapse collapse">
      <div class="panel-body"> 
      copy and paste the following code to the cell below, and run the cell: <br>
    
      df_data_1.describe()

  </div>

**Task 4: Run the cell below to check unqiue values for categorical columns.**

In [ ]:
for i in ["incident_severity", "insured_hobbies", "police_report_available", "fraud_reported", "auto_year"]:
    print(i + ":")
    print(df_data_1[i].unique())
    print("\n")

## 4. Data Preparation

**After review this output, we have a better understanding of what our data looks like. And for data preparation, we may want to change the cateogory "?"  in _police_report_available_ to "UNKNOWN" to make it more readable.** 

**Task 5: Run the cell below to change "?" to "UNKNOWN" for the _police_report_available_ column.**

In [ ]:
df_data_1['police_report_available'] = df_data_1['police_report_available'].apply(lambda x :'UNKNOWN' if x == "?" else x)

## 5. Exploratory Data Analysis(EDA)

In this section, we will further explore the data by drawing some visualizations. Here are some sample questions we want to answer through EDA:

1. How many fraudulent cases do we have in the dataset?
2. Which incident_severity level has highest fraud ratio?

### 5.1 How many fraudulent cases are there in the dataset?
**Task 6: Run the cell below to draw a bar chart of fraud_reported**.

In [ ]:
f, ax = plt.subplots(figsize=(20, 10))
sns.countplot('fraud_reported',data=df_data_1)
plt.title("Fraud Count")
plt.show()

Like most fraud datasets, our label distribution is skewed, aka. more non-fraudulent cases than fraudulent cases.

### 5.2 Which incident_severity level has highest fraud ratio?
**Task 7: Run the cell below to draw a mosaic plot for fraud_reported by incident_severity level**.

In [ ]:
props = {}
props[('Major Damage', 'Y')] = {'color': 'xkcd:rose pink'}
props[('Major Damage', 'N')] = {'color': 'xkcd:sage green'}

props[('Minor Damage', 'Y')] = {'color': 'xkcd:rose pink'}
props[('Minor Damage', 'N')] = {'color': 'xkcd:sage green'}

props[('Total Loss', 'Y')] = {'color': 'xkcd:rose pink'}
props[('Total Loss', 'N')] = {'color': 'xkcd:sage green'}

props[('Trivial Damage', 'Y')] = {'color': 'xkcd:rose pink'}
props[('Trivial Damage', 'N')] = {'color': 'xkcd:sage green'}

with mpl.rc_context():
    mpl.rc("figure", figsize=(20,10))
    mosaic(df_data_1,["incident_severity", "fraud_reported"], gap=0.01, properties=props, title = "Fraud Ratio by incident_severity level")
    plt.show()

It seems when sverity level = Major Damage, the fraud ratio is the highest.

## 6. Modeling

In this section we will starting building the random forest model to predict fraud.

**Task 8: Run the cell below to build the model. The comment will help you understand what each step is doing.**

In [ ]:
label = "fraud_reported"
catCols = ["incident_severity", "insured_hobbies", "police_report_available", "auto_year"]
numCols = ["months_as_customer", "age", "capital-loss", "vehicle_claim", "policy_annual_premium"]

# convert pandas dataframe to spark dataframe
spark_df = spark.createDataFrame(df_data_1)

print("\n*********" + "Split data into training and test sets" + "*********")

# split into training and test sets
trainingData, testData = spark_df.randomSplit([0.7, 0.3], seed = 824)
trainingData.cache()
testData.cache()
print("Sample Size: " + str(spark_df.count()))
print("Training set size: " + str(trainingData.count()))
print("Testing set size: " + str(testData.count()))
print("Distribution of target in trainingData is: \n ", trainingData.groupBy("fraud_reported").count().take(2))

print("\n*********" + "Building pipeline" + "*********")

stages = [] # stages in our Pipeline
for categoricalCol in catCols:
    # Category Indexing with StringIndexer
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    # Use OneHotEncoder to convert categorical variables into binary SparseVectors
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    # Add stages.  These are not run here, but will run all at once later on.
    stages += [stringIndexer, encoder]

# encode label   
label_stringIdx = StringIndexer(inputCol="fraud_reported", outputCol="label")
stages += [label_stringIdx]

# assemble feature vector
assemblerInputs = [c + "classVec" for c in catCols] + numCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

# add RandomForestClassifier to the pipeline
rf_model = RandomForestClassifier(labelCol="label", featuresCol="features")
stages += [rf_model]

# convert predicted label to string
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=['N', 'Y'])
stages += [labelConverter]

# construct pipeline
pipeline_build = Pipeline().setStages(stages)

print("\n*********" + "Model Training" + "*********")

# fit model 
model_fit = pipeline_build.fit(trainingData)


print("\n*********" + "Model Evaluation" + "*********")

predictions = model_fit.transform(testData)

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="label")

# Model Eval: Area under ROC vs. precision-recall curve
tp = predictions[(predictions.label==1) & (predictions.prediction == 1)].count()
tn = predictions[(predictions.label == 0) & (predictions.prediction==0)].count()
fp = predictions[(predictions.label == 0) & (predictions.prediction==1)].count()
fn = predictions[(predictions.label == 1) & (predictions.prediction == 0)].count()
r = float(tp)/(tp + fn)
p = float(tp) / (tp + fp)

print("True Positives:", tp)
print("True Negatives:", tn)
print("False Positives:", fp)
print("False Negatives:", fn)
print("Total:", predictions.count())
print("recall:", r)
print("precision:", p)
print("Accuracy:", (tp + tn)/predictions.count())
print('Done! areaUnderROC = {:.2f}.'.format(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"}))) 
print('Done! areaUnderPR = {:.2f}.'.format(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})))

## 7. Save output back to Object Storage

**Task 9: Follow the steps below and save output file to Object Storage.**

9.1 Click on the triple dot on the top right corner, click **Insert project token**, it will insert a cell **at the top of the notebook** containing the project token. **Run that cell**.

9.2 Run the cell below to save the prediction file to Object Storage.

<img src="https://github.com/CatherineCao2016/pics/raw/master/inserttoken.png" width="500" height="800" align="middle"/>

In [ ]:
pred_pd = predictions.toPandas()
project.save_data("pred_output.csv", pred_pd.to_csv(index = False))

Now you should see the **pred_output.csv** under the **Data Assets** tab in the project view.

## 8. Deploy Model to Watson Machine Learning

**Task 10: Follow the steps below to save model to Watson Machine Learning.**

10.1 Grab your WML credentials from IBM Cloud and copy and paste it in the cell below.   
10.2 Then run the two cells below.

In [ ]:
wml_credentials = {
  "apikey": "*********************",
  "iam_apikey_description": "*********************",
  "iam_apikey_name": "*********************",
  "iam_role_crn": "*********************",
  "iam_serviceid_crn": "*********************",
  "instance_id": "*********************",
  "url": "*********************"
}


In [ ]:
client = WatsonMachineLearningAPIClient(wml_credentials)

# configure model metadata
model_props = {
    client.repository.ModelMetaNames.NAME: "Insurance_Fraud_Detection",
    #client.repository.ModelMetaNames.TRAINING_DATA_REFERENCE: training_data_reference,
    client.repository.ModelMetaNames.EVALUATION_METHOD: "binary",
    client.repository.ModelMetaNames.EVALUATION_METRICS: [
        {
           "name": "areaUnderROC",
           "value": .6,
           "threshold": 0.5
        }
    ]
}

published_model_details = client.repository.store_model(model=model_fit, meta_props=model_props, training_data=trainingData, pipeline=pipeline_build)
model_uid = client.repository.get_model_uid(published_model_details)

If you did not see any error message till this point, you should see the **Insurance_Fraud_Detection** model under the Models tab in the project view. Now go back to the lab instruction for more interection with the model deployment. 

<img src="https://github.com/CatherineCao2016/pics/raw/master/wmlmodel.png" align="middle"/>